In [21]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

df1 = pd.read_csv("NSW_Road_Crash.csv")
df1.dropna()

df2 = pd.read_csv("Traffic_Volume_Viewer.csv")
mask = df2['classification_type'].isin(['LIGHT VEHICLES', 'HEAVY VEHICLES']) & ~df2['cardinal_direction_name'].str.contains('AND|BOTH') & ~df2['period'].isin(['ALL DAYS', 'WEEKDAYS','PUBLIC HOLIDAYS'])
df2_new = df2[mask]

df2.dropna()
df2_new.head()

names = df2_new['period'].unique()
print(names)

['AM PEAK' 'OFF PEAK' 'PM PEAK' 'WEEKENDS']


In [2]:
#function to change String format
def String_format(name):
    if isinstance(name, str):
        new_str = name.capitalize()
        return new_str

String_format("MUAZ")

def abreviation(word,l1,l2):
    #search for the word in l1 and return i index of l2
    i = 0
    while i < len(l1):
       if word == l1[i]:
           return l2[i]
       else:
           i += 1
           continue 

In [3]:
unique_street_types = df1['Street type'].unique()
unique_street_types_list = list(unique_street_types)
full_street_type = ['Road','Way','Highway','Drive','Parade','Street','Avenue','Expressway','0','Distributor','Parkway','Court','Lane','Place','Boulevard','Close','Circuit','Crescent','Terrace','Bypass','Mall','Trail','Promenade','Grove','Track','Square','Esplanade','Gardens','Glen','Route','Driveway','Causeway','Loop','Chase','Access','Row','Walk','0']

unique_times = df1['Two-hour intervals'].unique()
unique_times_list = list(unique_times)
time_periods = ['PM PEAK','OFF PEAK','PM PEAK','OFF PEAK','OFF-PEAK','OFF-PEAK','AM PEAK','AM PEAK','PM PEAK','OFF PEAK','OFF PEAK','OFF PEAK',nan,nan]

#Loop through the dataset
l1 = unique_street_types
l2 = full_street_type

formatted_rows = []
for index, row in df1.iterrows():
    if not pd.isnull(row["Street Crash"]) and not pd.isnull(row["Street type"]):
        formatted_string = String_format(row["Street Crash"]) + " " + abreviation(row["Street type"], l1, l2)
        formatted_rows.append(formatted_string)
    else:
        formatted_rows.append(None)
    
    if row['Day of week of crash'].isin(['Saturday','Sunday']): 
        day_period = 'WEEKENDS'
    else: 
        day_period = abreviation(row["Two-hour intervals"], unique_times_list, time_periods)
    
df1["road_name"] = formatted_rows
df1.head()

,Crash ID,Degree of crash,Degree of crash - detailed,Reporting year,Year of crash,Month of crash,Day of week of crash,Two-hour intervals,Street Crash,Street type,...,DCA supplement,First impact type,Key TU type,Other TU type,No. of traffic units involved,No. killed,No. seriously injured,No. moderately injured,No. minor-other injured,road_name
0,1122708.0,Fatal,Fatal,2017.0,2017.0,January,Monday,18:00 - 19:59,HOLLOWAYS,RD,...,Left,Rollover,Station wagon,NaN,1.0,1.0,0.0,0.0,0.0,Holloways Road
1,1122709.0,Fatal,Fatal,2017.0,2017.0,January,Monday,12:00 - 13:59,PUTTY,RD,...,NaN,Head-on,Motorcycle,Light truck,3.0,1.0,1.0,1.0,0.0,Putty Road
2,1122710.0,Fatal,Fatal,2017.0,2017.0,January,Tuesday,14:00 - 15:59,IRRIGATION,WAY,...,NaN,Rear end,Artic tanker,Car (sedan/hatch),3.0,2.0,0.0,1.0,0.0,Irrigation Way
3,1123942.0,Fatal,Fatal,2017.0,2017.0,January,Thursday,10:00 - 11:59,VARDYS,RD,...,NaN,Vehicle - Pedestrian,Semi-trailer,Pedestrian,2.0,1.0,0.0,1.0,0.0,Vardys Road
4,1123948.0,Fatal,Fatal,2017.0,2017.0,January,Saturday,12:00 - 13:59,PRINCES,HWY,...,NaN,Head-on,Car (sedan/hatch),4 wheel drive,2.0,1.0,2.0,3.0,0.0,Princes Highway


In [4]:
# Filter both crash and traffic volume dataframe based on Road Name Column
df_crash = df1[df1['road_name'].isin(df2['road_name'])]
df_traffic = df2[df2['road_name'].isin(df1['road_name'])]


print('Crash Data:',df1.shape[0])
print('Crash Data Filtered:',df_crash.shape[0])

print('Traffic Data:',df2.shape[0])
print('Traffic Data Filtered:',df_traffic.shape[0])

Crash Data: 101719
Crash Data Filtered: 35691
Traffic Data: 221530
Traffic Data Filtered: 148192


In [5]:
import time

start_time = time.time()

# Loop through each element in df_traffic
crash_count_list = []
for index, row in df_traffic.iterrows():
    # Filter crash dataframe to only include entries that allign with criteria
    crash_temp = df_crash[df_crash['road_name'] == row['road_name']]
    # Get number of entires and store as new list
    crash_count = crash_temp.shape[0]
    crash_count_list.append(crash_count)

df_traffic['crash_count'] = crash_count_list
df_traffic.head()

end_time = time.time()

time_taken = end_time - start_time
print('Time Taken:',time_taken)

# Take taken is 220 secconds = 3.7 mins. For criteria still need: cardinal direction*, year, period*, vehicle classification*

Time Taken: 375.1695761680603
